In [ ]:
import sys 
sys.path.append('../../')

In [2]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account JSON key file
SERVICE_ACCOUNT_FILE = 'creds/google__sa.json'

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and create the service
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('drive', 'v3', credentials=credentials)

# Replace with your Google Drive folder ID
folder_id = '1sfPFHkXYpKyY41V0pfz3Qw3k4VLy5Hvb'

# Query to get files from the folder
query = f"'{folder_id}' in parents"

# Specify the page size
page_size = 1000  # You can change this number as needed

# Initialize page token
page_token = None


all_files = []
# Loop through pages of results
while True:
    response = service.files().list(q=query,
                                    spaces='drive',
                                    fields='nextPageToken, files(id, name, webViewLink)',
                                    pageSize=page_size,
                                    pageToken=page_token).execute()

    # Print file names and URLs
    for file in response.get('files', []):
        all_files.append(file)
        print('Found file: %s (%s) %s' % (file.get('name'), file.get('id'), file.get('webViewLink')))

    # Update the page token
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break


Found file: physics__query__2.ipynb (1OCPyw0a7PEXGGMKiPaCb8hAl78uy6VTy) https://drive.google.com/file/d/1OCPyw0a7PEXGGMKiPaCb8hAl78uy6VTy/view?usp=drivesdk
Found file: queues__query__4.ipynb (1qyX57pRfjan5fxX4srMMgEUVmx5iwQW6) https://drive.google.com/file/d/1qyX57pRfjan5fxX4srMMgEUVmx5iwQW6/view?usp=drivesdk
Found file: economics__modification__0.ipynb (1Kpx0UQnc28tMFUyFYjr5VwpZuQzsyQ5w) https://drive.google.com/file/d/1Kpx0UQnc28tMFUyFYjr5VwpZuQzsyQ5w/view?usp=drivesdk
Found file: pythonic_code__query__2.ipynb (1eqEcqB4QHVJ78q5sa3g9GjyzwvF_dyBD) https://drive.google.com/file/d/1eqEcqB4QHVJ78q5sa3g9GjyzwvF_dyBD/view?usp=drivesdk
Found file: transfer_learning__query__3.ipynb (1KwHzEamkWh5Go3rB5Aja26RVZwinhKkz) https://drive.google.com/file/d/1KwHzEamkWh5Go3rB5Aja26RVZwinhKkz/view?usp=drivesdk
Found file: decorators__modification__2.ipynb (1dH1x59Sr3DmEOvNVw7K_Hlb3NVnUooih) https://drive.google.com/file/d/1dH1x59Sr3DmEOvNVw7K_Hlb3NVnUooih/view?usp=drivesdk
Found file: statistics__query_

In [3]:
all_files[:10]

[{'webViewLink': 'https://drive.google.com/file/d/1OCPyw0a7PEXGGMKiPaCb8hAl78uy6VTy/view?usp=drivesdk',
  'id': '1OCPyw0a7PEXGGMKiPaCb8hAl78uy6VTy',
  'name': 'physics__query__2.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1qyX57pRfjan5fxX4srMMgEUVmx5iwQW6/view?usp=drivesdk',
  'id': '1qyX57pRfjan5fxX4srMMgEUVmx5iwQW6',
  'name': 'queues__query__4.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1Kpx0UQnc28tMFUyFYjr5VwpZuQzsyQ5w/view?usp=drivesdk',
  'id': '1Kpx0UQnc28tMFUyFYjr5VwpZuQzsyQ5w',
  'name': 'economics__modification__0.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1eqEcqB4QHVJ78q5sa3g9GjyzwvF_dyBD/view?usp=drivesdk',
  'id': '1eqEcqB4QHVJ78q5sa3g9GjyzwvF_dyBD',
  'name': 'pythonic_code__query__2.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1KwHzEamkWh5Go3rB5Aja26RVZwinhKkz/view?usp=drivesdk',
  'id': '1KwHzEamkWh5Go3rB5Aja26RVZwinhKkz',
  'name': 'transfer_learning__query__3.ipynb'},
 {'webViewLink': 'https://drive.google.co

In [4]:
folder_id = all_files[0]['id']
colab_url = f"https://colab.research.google.com/drive/{folder_id}"

print("Colab URL:", colab_url)
print("Drive URL:", all_files[0]['webViewLink'])

Colab URL: https://colab.research.google.com/drive/1OCPyw0a7PEXGGMKiPaCb8hAl78uy6VTy
Drive URL: https://drive.google.com/file/d/1OCPyw0a7PEXGGMKiPaCb8hAl78uy6VTy/view?usp=drivesdk


In [5]:
all_colab_urls = []
for file in all_files:
    folder_id = file['id']
    colab_url = f"https://colab.research.google.com/drive/{folder_id}"
    all_colab_urls.append(colab_url)

In [6]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import nbformat

def download_and_extract_metadata(service_account_file, file_id):
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file, scopes=['https://www.googleapis.com/auth/drive'])
    service = build('drive', 'v3', credentials=credentials)

    # Request to download the file
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    # Download the file
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print("Download progress: %d%%." % int(status.progress() * 100))

    # Move the buffer's pointer to the beginning
    fh.seek(0)

    # Open the notebook
    notebook = nbformat.read(fh, as_version=4)

    # Extract the first cell
    first_cell = notebook.cells[0]

    lines = first_cell["source"].split("\n")
    metadata = {}
    for line in lines:
        if "**Python Topics**" in line:
            metadata["topic"] = line.split(" - ")[1]
        if "**Type**" in line:
            metadata["type"] = line.split(" - ")[1]
        if "**Target Number of Turns (User + Assistant)**" in line:
            metadata["target_turns"] = line.split(" - ")[1]

    return metadata


In [9]:
# Filter Out Released Conversations

from src.sheets_utils import download_sheet_as_df

df_b3 = download_sheet_as_df(service_account_path=SERVICE_ACCOUNT_FILE, sheet_id="1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4", sheet_name="Conversations_Batch_3")

b3_links = set(df_b3["task_link"].tolist())
all_colab_urls = [url for url in all_colab_urls if url not in b3_links]

In [10]:
len(all_colab_urls)

736

In [15]:
# Divide the URLs into batches of 335 (Randomly)
import random
random.shuffle(all_colab_urls)
batch_3 = all_colab_urls[:335]
batch_4 = all_colab_urls[335:670]
batch_5 = all_colab_urls[670:]
batches = [batch_3, batch_4, batch_5]

In [18]:
shared_already = [
    "https://colab.research.google.com/drive/1lHYB-8JiU67LlaqjvaRuLYUbetxWbnD5",
    "https://colab.research.google.com/drive/1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap",
    "https://colab.research.google.com/drive/1nJqAbuEAm7gGoJvAS6iFNjHWnWOqHpyk"
]

for batch in batches:
    for url in batch:
        if url in shared_already:
            # batch.pop(batch.index(url))
            print(url)

In [11]:
import threading

def process_file(url, service_account_file, data_batch):
    file_id = url.split("/")[-1]
    metadata = download_and_extract_metadata(service_account_file, file_id)
    data = {
        "task_link": url,
        "assigned_to_email": "",
        "completion_status": "Unclaimed",
        "modified_question": "",
        "duration_mins": "",
        "completion_date": "",
        "comments": "",
        "metadata__type": metadata["type"],
        "metadata__topic": metadata["topic"],
        "metadata__target_length": metadata["target_turns"],
    }
    data_batch.append(data)

# List to hold thread objects
threads = []

# List to hold processed data
data = []

for url in all_colab_urls:
    # Create a new thread for each file
    thread = threading.Thread(target=process_file, args=(url, SERVICE_ACCOUNT_FILE, data))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

# At this point, data_batch_3 contains all the processed data
data[:10]

Download progress: 100%.Download progress: 100%.

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


Exception in thread Thread-264:
Traceback (most recent call last):
  File "/home/joe96/miniconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/joe96/miniconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2956/2710292803.py", line 5, in process_file
  File "/tmp/ipykernel_2956/1936207454.py", line 28, in download_and_extract_metadata
  File "/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/nbformat/__init__.py", line 171, in read
    return reads(buf, as_version, capture_validation_error, **kwargs)
  File "/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/nbformat/__init__.py", line 89, in reads
    nb = reader.reads(s, **kwargs)
  File "/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/nbformat/reader.py", line 77, in reads
    (major, minor) = get_versio

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.Download progress: 100%.
Download progress: 100%.

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


[{'task_link': 'https://colab.research.google.com/drive/1lHYB-8JiU67LlaqjvaRuLYUbetxWbnD5',
  'assigned_to_email': '',
  'completion_status': 'Unclaimed',
  'modified_question': '',
  'duration_mins': '',
  'completion_date': '',
  'comments': '',
  'metadata__type': 'modification',
  'metadata__topic': 'algorithms > by_data_structure > trees',
  'metadata__target_length': '2+'},
 {'task_link': 'https://colab.research.google.com/drive/111X9gUFmLetFRlgzdkHVkjxP7mS5hTAK',
  'assigned_to_email': '',
  'completion_status': 'Unclaimed',
  'modified_question': '',
  'duration_mins': '',
  'completion_date': '',
  'comments': '',
  'metadata__type': 'query',
  'metadata__topic': 'web_development > web_servers',
  'metadata__target_length': '2+'},
 {'task_link': 'https://colab.research.google.com/drive/18PLqn7o7-OZ39IfF2ZeBR8nBstRmV6w3',
  'assigned_to_email': '',
  'completion_status': 'Unclaimed',
  'modified_question': '',
  'duration_mins': '',
  'completion_date': '',
  'comments': '',
  

In [13]:
import pandas as pd

df = pd.DataFrame(data)
df

task_link  ... metadata__target_length
0    https://colab.research.google.com/drive/1lHYB-...  ...                      2+
1    https://colab.research.google.com/drive/111X9g...  ...                      2+
2    https://colab.research.google.com/drive/18PLqn...  ...                      2+
3    https://colab.research.google.com/drive/1efq1S...  ...                      2+
4    https://colab.research.google.com/drive/1Q1C_b...  ...                      2+
..                                                 ...  ...                     ...
730  https://colab.research.google.com/drive/1Owlkj...  ...                      2+
731  https://colab.research.google.com/drive/1m9Ti2...  ...                      2+
732  https://colab.research.google.com/drive/1J3EkE...  ...                      2+
733  https://colab.research.google.com/drive/1FVFH6...  ...                      2+
734  https://colab.research.google.com/drive/1wVy7J...  ...                      2+

[735 rows x 10 columns]

In [ ]:
from src.sheets_utils import upload_df_to_sheet

upload_df_to_sheet(service_account_path=SERVICE_ACCOUNT_FILE, sheet_id="1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4", sheet_name="Conversations_Batch_4", df=df)

In [22]:
df.to_csv("batch_3.csv", index=False)

In [30]:
df_3 = pd.read_csv("batch_3.csv")
df_3

0
0    https://colab.research.google.com/drive/1T_Jkg...
1    https://colab.research.google.com/drive/13OnWs...
2    https://colab.research.google.com/drive/1xTwo8...
3    https://colab.research.google.com/drive/1s6QrF...
4    https://colab.research.google.com/drive/1uwo3Y...
..                                                 ...
328  https://colab.research.google.com/drive/1aNfPg...
329  https://colab.research.google.com/drive/1QhKUP...
330  https://colab.research.google.com/drive/1yd4hr...
331  https://colab.research.google.com/drive/1CQSKk...
332  https://colab.research.google.com/drive/1VmX6t...

[333 rows x 1 columns]

In [35]:
df_3.merge(df, left_on="0", right_on="task_link", how="inner").to_csv("batch_3_all_data.csv", index=False)